In [10]:
# TRAIN_OR_VALIDATION = 'TRAIN'
TRAIN_OR_VALIDATION = 'VALIDATION'

In [2]:
import json
import pandas as pd

In [12]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

# input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_validation = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_DEV_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

# top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_2023.12.11.pickle"
# top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

# output_data_path = base_path + "/../Mistral/data/output/" + experiment_name
# log_filename = base_path + "/../Mistral/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [13]:
if TRAIN_OR_VALIDATION == 'TRAIN':
  data = pd.read_excel(input_data_path_train) #[['fine_tuning_prompt']].rename(columns={'fine_tuning_prompt': 'text'})
elif TRAIN_OR_VALIDATION == 'VALIDATION':
  data = pd.read_excel(input_data_path_validation) #[['fine_tuning_prompt']].rename(columns={'fine_tuning_prompt': 'text'})
else:
  raise Exception("ERRORE!!! IL VALORE DI TRAIN_OR_VALIDATION PUO' ESSERE SOLO TRAIN O VALIDATION")
data.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,0,5545,NaN,"Murray revealed on Sky Sports, “I hope not, bu...",0,NaN,NaN,NaN,145
1,1,32,3155,NaN,"With age and inactive lifestyle, your joints a...",0,NaN,NaN,NaN,100
2,2,33,2441,NaN,(Reuters) -Home health and hospice caregiver A...,0,NaN,NaN,NaN,140


In [14]:
def apply_fine_tuning_template(row):

  input_text = row['text']
  label = "---SDG-"+str(row['sdg'])+"---"

  zero_shot_instruction = f"""<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG-0---".

DO NOT ABSOLUTELY INCLUDE FOR ANY REASON THE INPUT TEXT IN THE OUTPUT!

Answer with label only using the format:---LABEL---.

The text to be processed is:
<</SYS>>

'''
{input_text}
'''
"""

  instruction = zero_shot_instruction

  template = f"""<s>[INST] {instruction} [/INST] "{label}" </s>"""

  return template

In [15]:
# @title
# def apply_fine_tuning_template(data):
#   text = data.text
#   label = data.sdg

#   template = f"""<|system|>\nYou are an helpful virtual assistant specialized in a multi-class single-label text classification task related to the domain of the Sustainable Development Goals.</s>\n<|user|>\nClassify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

# "SDG-1": "End poverty in all its forms everywhere."
# "SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
# "SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
# "SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
# "SDG-5": "Achieve gender equality and empower all women and girls."
# "SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
# "SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
# "SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
# "SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
# "SDG-10": "Reduce inequality within and among countries."
# "SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
# "SDG-12": "Ensure sustainable consumption and production patterns."
# "SDG-13": "Take urgent action to combat climate change and its impacts."
# "SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
# "SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
# "SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
# "SDG-0": "Other."

# Choose ONLY ONE label for each input text.

# DO NOT include the input text in your answer.

# The input text is:
# '''
# {text}
# '''</s>\n<|assistant|>\n"SDG-{label}"
# """

#   return template


In [16]:
data['fine_tuning_prompt'] = data.apply(apply_fine_tuning_template, axis=1)

In [17]:
data.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,fine_tuning_prompt
0,0,0,5545,NaN,"Murray revealed on Sky Sports, “I hope not, bu...",0,NaN,NaN,NaN,145,<s>[INST] <<SYS>>\nClassify the following text...
1,1,32,3155,NaN,"With age and inactive lifestyle, your joints a...",0,NaN,NaN,NaN,100,<s>[INST] <<SYS>>\nClassify the following text...
2,2,33,2441,NaN,(Reuters) -Home health and hospice caregiver A...,0,NaN,NaN,NaN,140,<s>[INST] <<SYS>>\nClassify the following text...


In [18]:
print(data.iloc[0]['fine_tuning_prompt'])

<s>[INST] <<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG-0---".

DO NOT ABSOLUTELY INCLUDE FOR ANY REASON THE INPUT TEXT IN THE OUTPUT!

Answer with label only using the format:---LABEL---.

The text to be processed is:
<</SYS>>

'''
Murray revealed on Sky Sports, “I hope not, but yo

In [19]:
texts = [{'text': text} for text in data['fine_tuning_prompt']]

In [20]:
texts[0]

{'text': '<s>[INST] <<SYS>>\nClassify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.\n\nIn the following lines, a text enclosed in triple quotes will be given to you.\n\nThe task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.\n\nThe constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.\n\nIf you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG-0---".\n\nDO NOT ABSOLUTELY INCLUDE FOR ANY REASON THE INPUT TEXT IN THE OUTPUT!\n\nAnswer with label only using the format:---LABEL---.\n\nThe text to be processed is:\n<</SYS>>\n\n\'\'\'\nMurray revealed on

In [21]:
# with open('/content/gdrive/MyDrive/Colab/SDG/Mistral/Zephyr_fine_tuning_prompt.jsonl', 'w') as file:
#     for t in texts:
#       json.dump(t, file, ensure_ascii=False)

In [22]:
if TRAIN_OR_VALIDATION == 'TRAIN':
  with open('/content/gdrive/MyDrive/Colab/SDG/Llama-2_FineTuning/Llama-2_fine_tuning_prompt_4_TogetherAI.jsonl', 'w') as file:
      for text in texts:
          file.write(json.dumps(text) + '\n')
elif TRAIN_OR_VALIDATION == 'VALIDATION':
  with open('/content/gdrive/MyDrive/Colab/SDG/Llama-2_FineTuning/Llama-2_fine_tuning_prompt_4_TogetherAI_VALIDATION.jsonl', 'w') as file:
      for text in texts:
          file.write(json.dumps(text) + '\n')
else:
  raise Exception("ERRORE!!! IL VALORE DI TRAIN_OR_VALIDATION PUO' ESSERE SOLO TRAIN O VALIDATION")

